In [1]:
import pandas as pd
import sys
import json
import LLDA.model.labeled_lda as llda
import numpy as np
from sklearn.metrics import pairwise_distances
from scipy.spatial.distance import jensenshannon

# sys.path.append('../')

In [2]:
df = pd.read_csv('FRIS_projects_pubs.csv')
dis_list = df['disciplines'].values
topics = []
for dis in dis_list:
    dis = dis.replace("[", "").replace("]", "").replace("'", "").replace(" ", "").split(',')
    dis = [d.strip() for d in dis]
    topics.extend(dis)

topics = sorted(list(set(topics)))
print(topics)
len(topics)

['0101', '0102', '0103', '0104', '0105', '0106', '0107', '0199', '0201', '0202', '0203', '0204', '0205', '0206', '0207', '0208', '0299', '0301', '0302', '0303', '0304', '0305', '0306', '0399', '0401', '0402', '0499', '0501', '0502', '0503', '0504', '0505', '0506', '0507', '0508', '0599', '0601', '0602', '0603', '0604', '0699']


41

In [34]:
with open('FRIS_data.json') as f:
    data = json.load(f)

clean_data = {}
for pro_id in data.keys():
    if data[pro_id]['publications'] and data[pro_id]['disciplines']:
        clean_data[pro_id] = data[pro_id]
print(len(clean_data))

pro_id_list = list(clean_data.keys())
for pro_id in pro_id_list:
    pubs = {}
    pro_dis_list = []
    for pub_id in clean_data[pro_id]['publications'].keys():
        if clean_data[pro_id]['publications'][pub_id]['disciplines'] and clean_data[pro_id]['publications'][pub_id]['authors']:
            pro_dis_list.extend(clean_data[pro_id]['publications'][pub_id]['disciplines'])
            pubs[pub_id] = clean_data[pro_id]['publications'][pub_id]
    
    if pubs:
        clean_data[pro_id]['publications'] = pubs
        clean_data[pro_id]['disciplines'] = list(set(pro_dis_list))
    else:
        del clean_data[pro_id]

print(len(clean_data))

444
147


In [3]:
# read json files
with open('FRIS_projects_pubs_abstracts.json') as f:
    abstract_data = json.load(f)

with open('FRIS_projects_pubs_dis.json') as f:
    dis_data = json.load(f)

abstract_data_keys = [list(entry.keys())[0] for entry in abstract_data]
project_data = []
pub_data = []

data = []
for dis_entry in dis_data:
    dis_entry = dict(dis_entry)
    key = list(dis_entry.keys())[0]

    abs_entry = None
    for ent in abstract_data:
        ent = dict(ent)
        if key == list(ent.keys())[0]:
            abs_entry = ent
            break
    
    if not abs_entry:
        continue

    intersection = [x for x in abs_entry[key]['pub_ids'] if x in frozenset(dis_entry[key]['pub_ids'])]
    if not intersection:
        continue

    project_abstract = abs_entry[key]['project_abstract']
    project_dis = dis_entry[key]['project_disciplines']

    abstract_idx = [abs_entry[key]['pub_ids'].index(x) for x in intersection]
    pub_abstracts = [abs_entry[key]['pub_abstracts'][i] for i in abstract_idx]
    dis_idx = [dis_entry[key]['pub_ids'].index(x) for x in intersection]
    pub_dis = [dis_entry[key]['pub_disciplines'][i] for i in dis_idx]

    data.append([key, project_abstract, project_dis, intersection, pub_abstracts, pub_dis])

split = int(len(data) * 0.8)
train_data = data[:split]
test_data = data[split:]

train_project_data = [(entry[1], entry[2]) for entry in train_data]
train_pub_data = []
for entry in train_data:
    for i in range(len(entry[3])):
        train_pub_data.append((entry[4][i], entry[5][i]))

test_project_data = [(entry[1], entry[2]) for entry in test_data]
test_pub_data = []
for entry in test_data:
    for i in range(len(entry[3])):
        test_pub_data.append((entry[4][i], entry[5][i]))

In [4]:
dis = []
for i in data:
    dis.extend(i[2])
pro_topic = set(dis)
print(len(pro_topic))

dis_list = []
for i in data:
    dis_list.extend(i[5])
dis = []
for i in dis_list:
    dis.extend(i)
pub_topic = set(dis)
print(len(pub_topic))

topic_intersection = pro_topic.intersection(pub_topic)
len(topic_intersection)

35
34


32

In [5]:
data_new = []
for entry in data:
    project_dis = entry[2]
    pub_ids = entry[3]
    pub_abstracts = entry[4]
    pub_dis = entry[5]
    pub_dis_new = []
    pub_ids_new = []
    for i, dis_list in enumerate(pub_dis):
        if set(dis_list).issubset(topic_intersection):
            pub_dis_new.append(dis_list)
            pub_ids_new.append(entry[3][i])
    
    if not (set(project_dis).issubset(topic_intersection) and pub_dis_new):
        continue

    pub_idx = [pub_ids.index(x) for x in pub_ids_new]
    pub_abstracts_new = [pub_abstracts[i] for i in pub_idx]
    entry[3] = pub_ids_new
    entry[4] = pub_abstracts_new
    entry[5] = pub_dis_new

    data_new.append(entry)

split = int(len(data_new) * 0.8)
train_data = data_new[:split]
test_data = data_new[split:]

train_project_data = [(entry[1], entry[2]) for entry in train_data]
train_pub_data = []
for entry in train_data:
    for i in range(len(entry[3])):
        train_pub_data.append((entry[4][i], entry[5][i]))

test_project_data = [(entry[1], entry[2]) for entry in test_data]
test_pub_data = []
for entry in test_data:
    for i in range(len(entry[3])):
        test_pub_data.append((entry[4][i], entry[5][i]))

In [6]:
dis = []
for i in data_new:
    dis.extend(i[2])
pro_topic = set(dis)
print(len(pro_topic))

dis_list = []
for i in data_new:
    dis_list.extend(i[5])
dis = []
for i in dis_list:
    dis.extend(i)
pub_topic = set(dis)
print(len(pub_topic))

topic_intersection = pro_topic.intersection(pub_topic)
len(topic_intersection)

32
31


31

In [7]:
data_new_new = []
for entry in data_new:
    project_dis = entry[2]
    pub_ids = entry[3]
    pub_abstracts = entry[4]
    pub_dis = entry[5]
    pub_dis_new = []
    pub_ids_new = []
    for i, dis_list in enumerate(pub_dis):
        if set(dis_list).issubset(topic_intersection):
            pub_dis_new.append(dis_list)
            pub_ids_new.append(entry[3][i])
    
    if not (set(project_dis).issubset(topic_intersection) and pub_dis_new):
        continue

    pub_idx = [pub_ids.index(x) for x in pub_ids_new]
    pub_abstracts_new = [pub_abstracts[i] for i in pub_idx]
    entry[3] = pub_ids_new
    entry[4] = pub_abstracts_new
    entry[5] = pub_dis_new

    data_new_new.append(entry)

split = int(len(data_new_new) * 0.95)
train_data = data_new_new[:split]
test_data = data_new_new[split:]

train_project_data = [(entry[1], entry[2]) for entry in train_data]
train_pub_data = []
for entry in train_data:
    for i in range(len(entry[3])):
        train_pub_data.append((entry[4][i], entry[5][i]))

test_project_data = [(entry[1], entry[2]) for entry in test_data]
test_pub_data = []
for entry in test_data:
    for i in range(len(entry[3])):
        test_pub_data.append((entry[4][i], entry[5][i]))

In [8]:
dis = []
for i in data_new_new:
    dis.extend(i[2])
pro_topic = set(dis)
print(len(pro_topic))

dis_list = []
for i in data_new_new:
    dis_list.extend(i[5])
dis = []
for i in dis_list:
    dis.extend(i)
pub_topic = set(dis)
print(len(pub_topic))

topic_intersection = pro_topic.intersection(pub_topic)
len(topic_intersection)

31
31


31

In [9]:
print('Train project data:', train_project_data[0])
print('Train pub data:', train_pub_data[0])

Train project data: ('Sensorineural hearing loss (SNHL) affects one out of three older than 65, yet imaging methods have only recently become sensitive enough to isolate the specific elements which constitute SNHL (e.g. synaptic and hair-cell damage). Because these elements have different functional roles for hearing, their damage needs to be individually characterized and taken into account in therapeutic interventions. Differential diagnosis is possible in controlled animal experiments via post-mortem histology, but is much harder to achieve in humans where unknown mixtures of SNHL need to be quantified non-invasively.\nTo translate findings from the animal domain to humans, we adopt a computational model of the auditory periphery which integrates the histopathology and physiology of hearing to predict how human physiological responses are affected by SNHL. We focus on simultaneously modeling the neural generators of auditory evoked potentials, otoacoustic emissions and the middle-ea

In [10]:
print(len(data_new_new))
print(len(train_project_data), len(train_pub_data))
print(len(test_project_data), len(test_pub_data))

154
146 513
8 64


In [11]:
# # load from disk
# llda_model_project = llda.LldaModel()
# llda_model_project.load_model_from_dir("LLDA/data_models/pro_abstracts_disciplines", load_derivative_properties=False)
# print("llda_model_project", llda_model_project)
# # print("Top-5 terms of topic 'virus': ", llda_model_project.top_terms_of_topic("virus", 5, False))
# print("Doc-Topic Matrix: \n", llda_model_project.theta)
# print("Topic-Term Matrix: \n", llda_model_project.beta)

In [12]:
train_project_copy = train_project_data.copy()
train_pub_copy = train_pub_data.copy()

In [13]:
llda_model_project = llda.LldaModel(labeled_documents=train_project_copy)
print(llda_model_project)
llda_model_pub = llda.LldaModel(labeled_documents=train_pub_copy)
print(llda_model_pub)


Labeled-LDA Model:
	K = 32
	M = 146
	T = 7084
	WN = 27783
	LN = 253
	alpha = 1.5625
	eta = 0.001
	perplexity = 626.3098551831756
	

Labeled-LDA Model:
	K = 32
	M = 513
	T = 17841
	WN = 102055
	LN = 1147
	alpha = 1.5625
	eta = 0.001
	perplexity = 1199.36248292791
	


In [14]:
# training
llda_model_project.training(iteration=10, log=True)
# while True:
#     print("iteration %s sampling..." % (llda_model.iteration + 1))
#     llda_model.training(1)
#     print("after iteration: %s, perplexity: %s" % (llda_model.iteration, llda_model.perplexity()))
#     print("delta beta: %s" % llda_model.delta_beta)
#     if llda_model.is_convergent(method="beta", delta=0.01):
#         break

after iteration: 0, perplexity: 626.3098551831756
gibbs sample count:  27783
after iteration: 1, perplexity: 537.9745709990104
gibbs sample count:  27783
after iteration: 2, perplexity: 530.3084066511476
gibbs sample count:  27783
after iteration: 3, perplexity: 533.5084783041957
gibbs sample count:  27783
after iteration: 4, perplexity: 534.1111164759184
gibbs sample count:  27783
after iteration: 5, perplexity: 535.6866421315981
gibbs sample count:  27783
after iteration: 6, perplexity: 537.6771979380878
gibbs sample count:  27783
after iteration: 7, perplexity: 538.0678367162153
gibbs sample count:  27783
after iteration: 8, perplexity: 537.3626278738654
gibbs sample count:  27783
after iteration: 9, perplexity: 539.5771113976285
gibbs sample count:  27783


In [15]:
llda_model_pub.training(iteration=10, log=True)

after iteration: 0, perplexity: 1199.36248292791
gibbs sample count:  102055
after iteration: 1, perplexity: 1087.5740881499812
gibbs sample count:  102055
after iteration: 2, perplexity: 1067.8215730019954
gibbs sample count:  102055
after iteration: 3, perplexity: 1064.3029035619495
gibbs sample count:  102055
after iteration: 4, perplexity: 1062.0042117761184
gibbs sample count:  102055
after iteration: 5, perplexity: 1059.4585316258572
gibbs sample count:  102055
after iteration: 6, perplexity: 1059.196811247672
gibbs sample count:  102055
after iteration: 7, perplexity: 1057.7283022565457
gibbs sample count:  102055
after iteration: 8, perplexity: 1057.4111038365454
gibbs sample count:  102055
after iteration: 9, perplexity: 1057.275269868792
gibbs sample count:  102055


In [16]:
# # update
# print("before updating: ", llda_model)
# update_labeled_documents = [("new example test example test example test example test", ["example", "test"])]
# llda_model.update(labeled_documents=update_labeled_documents)
# print("after updating: ", llda_model)

# # train again
# # llda_model.training(iteration=10, log=True)
# while True:
#     print("iteration %s sampling..." % (llda_model.iteration + 1))
#     llda_model.training(1)
#     print("after iteration: %s, perplexity: %s" % (llda_model.iteration, llda_model.perplexity()))
#     print("delta beta: %s" % llda_model.delta_beta)
#     if llda_model.is_convergent(method="beta", delta=0.01):
#         break

In [17]:
# inference
# note: the result topics may be different for difference training, because gibbs sampling is a random algorithm
document = test_project_data[0][0]
print(document)

topics = llda_model_project.inference(document=document, iteration=100, times=10)
# print sorted topics
sorted(topics, key=lambda x: x[0])

Computer models have become an important tool to understand causal mechanisms underlying human movement. Simulating realistic musculoskeletal dynamics is critical to understanding neural control of muscle activity and motor performance. State-of-the-art simulations of motion are based on phenomenological muscle-tendon models that do not take into account the history dependence of muscle action. However, force enhancement and depression following dynamic contractions measured in isolated muscle fibers may be as high as 50% of the corresponding steady-state isometric force. The aim of the proposed project is therefore to investigate the contribution of history dependent muscle action to dynamic whole-body human movements by combining experimental data and computational modeling. We hypothesize that history-dependent muscle action contributes substantially to stability of walking in the presence of external perturbations and increases performance of sports movements such as jumping and ru

[('0101', 0.010812883435582822),
 ('0102', 0.012039877300613497),
 ('0103', 0.022469325153374235),
 ('0104', 0.017561349693251533),
 ('0105', 0.05375766871165644),
 ('0106', 0.03412576687116564),
 ('0107', 0.010199386503067485),
 ('0201', 0.02492331288343558),
 ('0202', 0.010812883435582822),
 ('0203', 0.02062883435582822),
 ('0204', 0.01142638036809816),
 ('0205', 0.0169478527607362),
 ('0206', 0.010812883435582822),
 ('0207', 0.010199386503067485),
 ('0208', 0.023696319018404906),
 ('0299', 0.02492331288343558),
 ('0301', 0.027377300613496934),
 ('0302', 0.020015337423312886),
 ('0303', 0.022469325153374235),
 ('0304', 0.012039877300613497),
 ('0305', 0.017561349693251533),
 ('0306', 0.019401840490797545),
 ('0401', 0.02553680981595092),
 ('0402', 0.009585889570552147),
 ('0501', 0.02799079754601227),
 ('0502', 0.010812883435582822),
 ('0504', 0.009585889570552147),
 ('0507', 0.012039877300613497),
 ('0602', 0.010812883435582822),
 ('0603', 0.0169478527607362),
 ('0604', 0.0108128834

In [18]:
documents = test_project_data
topic_matrix_project = np.zeros((len(documents), llda_model_project.K))

for i, (document, _) in enumerate(documents):
    topics = llda_model_project.inference(document=document, iteration=100, times=10)
    sorted_topics = sorted(topics, key=lambda x: x[0])
    for j, (topic, prob) in enumerate(sorted_topics):
        topic_matrix_project[i, j] = prob

topic_matrix_project

array([[0.00958589, 0.01388037, 0.02369632, 0.01940184, 0.02308282,
        0.03167178, 0.01081288, 0.0279908 , 0.01203988, 0.01878834,
        0.01203988, 0.01572086, 0.01019939, 0.01142638, 0.02001534,
        0.01817485, 0.03535276, 0.02369632, 0.01633436, 0.01081288,
        0.01572086, 0.05130368, 0.01633436, 0.01203988, 0.02308282,
        0.01019939, 0.01081288, 0.01326687, 0.01081288, 0.01388037,
        0.01019939, 0.4476227 ],
       [0.0117515 , 0.01115269, 0.02013473, 0.01414671, 0.01474551,
        0.0117515 , 0.00995509, 0.02372754, 0.02432635, 0.02133234,
        0.01115269, 0.0129491 , 0.00935629, 0.00995509, 0.00935629,
        0.01474551, 0.03929641, 0.0380988 , 0.03510479, 0.02612275,
        0.01893713, 0.02851796, 0.0129491 , 0.05007485, 0.01654192,
        0.00995509, 0.01773952, 0.01474551, 0.01115269, 0.00995509,
        0.02372754, 0.41654192],
       [0.00737028, 0.00878538, 0.01727594, 0.01916274, 0.00737028,
        0.01774764, 0.01114387, 0.01208726, 0.0668

In [19]:
documents = test_pub_data
topic_matrix_pub = np.zeros((len(documents), llda_model_pub.K))

for i, (document, _) in enumerate(documents):
    topics = llda_model_pub.inference(document=document, iteration=100, times=10)
    sorted_topics = sorted(topics, key=lambda x: x[0])
    for j, (topic, prob) in enumerate(sorted_topics):
        topic_matrix_pub[i, j] = prob

topic_matrix_pub

array([[0.00785   , 0.00825   , 0.00865   , ..., 0.01185   , 0.01585   ,
        0.49385   ],
       [0.02000689, 0.01008953, 0.0089876 , ..., 0.00568182, 0.00457989,
        0.3952135 ],
       [0.00858696, 0.00597826, 0.01293478, ..., 0.00945652, 0.0198913 ,
        0.49496377],
       ...,
       [0.01127049, 0.0101776 , 0.0364071 , ..., 0.03203552, 0.11181694,
        0.34296448],
       [0.01578664, 0.00975216, 0.01794181, ..., 0.00802802, 0.00845905,
        0.40070043],
       [0.02516026, 0.02003205, 0.03285256, ..., 0.02003205, 0.02644231,
        0.05849359]])

In [20]:
# perplexity
# calculate perplexity on test data
perplexity = llda_model_project.perplexity(documents=map(lambda x: x[0], test_project_data),
                                   iteration=30,
                                   times=10)
print("perplexity on test data: %s" % perplexity)
# calculate perplexity on training data
print("perplexity on training data: %s" % llda_model_project.perplexity())

perplexity on test data: 851.9515738134049
perplexity on training data: 540.1862195263787


In [21]:
# perplexity
# calculate perplexity on test data
perplexity = llda_model_pub.perplexity(documents=map(lambda x: x[0], test_pub_data),
                                   iteration=30,
                                   times=10)
print("perplexity on test data: %s" % perplexity)
# calculate perplexity on training data
print("perplexity on training data: %s" % llda_model_pub.perplexity())

perplexity on test data: 1459.1165750361565
perplexity on training data: 1056.7377561158273


In [22]:
# save to disk
save_model_dir = "LLDA/data_models/topic_pro_disciplines"
# llda_model.save_model_to_dir(save_model_dir, save_derivative_properties=True)
llda_model_project.save_model_to_dir(save_model_dir)
# save to disk
save_model_dir = "LLDA/data_models/topic_pub_disciplines"
# llda_model.save_model_to_dir(save_model_dir, save_derivative_properties=True)
llda_model_pub.save_model_to_dir(save_model_dir)

In [23]:
jensenshannon(topic_matrix_project[0], topic_matrix_pub[0])

0.2123314748621048

In [24]:
distance_matrix = np.zeros((len(topic_matrix_project), len(topic_matrix_pub)))

for i in range(len(topic_matrix_project)):
    for j in range(len(topic_matrix_pub)):
        distance_matrix[i, j] = jensenshannon(topic_matrix_project[i], topic_matrix_pub[j])

In [25]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.DataFrame(distance_matrix)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63
0,0.212331,0.269175,0.267569,0.445697,0.295521,0.201339,0.223425,0.424491,0.376280,0.244440,0.170431,0.318744,0.227287,0.291724,0.303605,0.174852,0.196058,0.357871,0.212024,0.223233,0.260043,0.283683,0.291394,0.238238,0.161269,0.270053,0.212523,0.279449,0.230363,0.246146,0.194323,0.267119,0.205868,0.203038,0.180197,0.191759,0.221595,0.343878,0.272553,0.295849,0.215019,0.247162,0.257373,0.186615,0.231500,0.240552,0.198433,0.282268,0.254371,0.181228,0.288842,0.182244,0.192192,0.163516,0.305225,0.203716,0.223856,0.192887,0.201121,0.187965,0.266233,0.200574,0.140742,0.356169
1,0.231136,0.298907,0.262725,0.432929,0.308049,0.216204,0.197784,0.415110,0.391351,0.240240,0.188680,0.320688,0.243810,0.285619,0.269250,0.184387,0.208006,0.350881,0.191898,0.233026,0.255561,0.295676,0.249022,0.209673,0.132081,0.256331,0.216692,0.256276,0.195544,0.229222,0.216700,0.230112,0.180935,0.190366,0.175983,0.176993,0.182422,0.318054,0.269011,0.274085,0.196611,0.244016,0.244425,0.198950,0.239333,0.262293,0.218032,0.256241,0.219834,0.214002,0.264511,0.233603,0.223005,0.178004,0.285513,0.223407,0.216813,0.229262,0.237625,0.196559,0.254240,0.200791,0.199554,0.352770
2,0.262093,0.345770,0.297904,0.459758,0.338451,0.264828,0.300378,0.445076,0.406316,0.314303,0.221573,0.378324,0.315817,0.328758,0.359592,0.234488,0.281701,0.382778,0.284271,0.288223,0.317382,0.361321,0.334612,0.308704,0.225072,0.336785,0.314870,0.352075,0.296104,0.318435,0.312355,0.345496,0.252117,0.276095,0.240099,0.275102,0.280114,0.406421,0.337973,0.366031,0.292938,0.308544,0.319961,0.258929,0.323347,0.309847,0.281418,0.335834,0.302107,0.262799,0.369576,0.282436,0.280703,0.215390,0.329897,0.285541,0.291374,0.271476,0.284450,0.233440,0.335295,0.264292,0.259811,0.431062
3,0.194030,0.303330,0.266287,0.440226,0.305398,0.201898,0.270895,0.414908,0.391861,0.283398,0.153417,0.357203,0.283488,0.296868,0.341593,0.160076,0.236565,0.358383,0.264940,0.246965,0.281548,0.326550,0.308143,0.293788,0.205594,0.310957,0.296530,0.339363,0.288350,0.295935,0.274034,0.320125,0.235447,0.245809,0.205408,0.240509,0.267282,0.397039,0.331096,0.357198,0.285530,0.280128,0.285723,0.204507,0.313710,0.265971,0.253067,0.324404,0.274675,0.191375,0.365994,0.214107,0.256395,0.140820,0.275247,0.245669,0.250852,0.234797,0.235064,0.194030,0.329234,0.246028,0.225202,0.439002
4,0.192167,0.284199,0.263752,0.423042,0.266588,0.204078,0.226005,0.401792,0.363075,0.257674,0.173032,0.319419,0.266170,0.263199,0.294908,0.185194,0.184006,0.316555,0.215976,0.212271,0.230081,0.259868,0.270700,0.215968,0.183735,0.256267,0.214428,0.294768,0.240020,0.242160,0.264770,0.292781,0.221711,0.196552,0.178453,0.209296,0.235989,0.331807,0.267607,0.298760,0.205708,0.251692,0.237645,0.189748,0.296389,0.239090,0.250531,0.306992,0.254307,0.193334,0.328848,0.177173,0.246042,0.171403,0.306080,0.181027,0.210450,0.190480,0.195803,0.198598,0.313906,0.207109,0.194303,0.353991
5,0.227421,0.330559,0.291175,0.440144,0.320245,0.215985,0.264471,0.423508,0.392776,0.285099,0.206900,0.361661,0.291661,0.302138,0.323407,0.231093,0.223998,0.357871,0.190520,0.222842,0.285719,0.320884,0.288319,0.236260,0.207287,0.297178,0.230077,0.315725,0.236395,0.293558,0.274675,0.315549,0.187002,0.236686,0.221168,0.221981,0.255075,0.354142,0.300190,0.309966,0.261817,0.250147,0.287619,0.218483,0.288996,0.263976,0.229589,0.286892,0.281306,0.216667,0.320949,0.232185,0.160740,0.194159,0.347870,0.231698,0.259296,0.225539,0.258421,0.214001,0.264383,0.244474,0.184209,0.363463
6,0.237743,0.318926,0.261430,0.450897,0.307259,0.216096,0.251892,0.425919,0.374492,0.284840,0.199962,0.340818,0.262285,0.301872,0.322134,0.192754,0.224051,0.364510,0.234410,0.235309,0.270007,0.328255,0.296902,0.245436,0.186654,0.286576,0.269328,0.304774,0.266121,0.274235,0.214434,0.289099,0.179917,0.211176,0.220942,0.215163,0.257677,0.348508,0.304